# AutoEncoder

## Import Libraries

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models, losses
from tensorflow.keras.models import Sequential
from keras.models import Model
import matplotlib.pyplot as plt
from sklearn import metrics
import pandas as pd
from tensorflow.keras import optimizers
import os
import shutil

## Connect to Drive

In [2]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


## Set parameters

In [3]:
FEATURES_FOLDER = '/content/drive/My Drive/CV_Birds/features'
LOCAL_MODELS_FOLDER = "/content/models"
if not os.path.exists(LOCAL_MODELS_FOLDER):
  os.makedirs(LOCAL_MODELS_FOLDER)
GLOBAL_MODELS_FOLDER = "/content/drive/My Drive/CV_Birds/models/AutoEncoder"

BATCH_SIZE = 256

In [4]:
callbacks_list = [
    keras.callbacks.EarlyStopping(
        monitor='val_loss',
        patience=6,
        restore_best_weights=True,
    ),
    keras.callbacks.ModelCheckpoint(
        filepath='autoencoder.keras',
        monitor='val_loss',
        save_best_only=True,
    )
]

For Finetuning:

In [ ]:
training_features = np.load(FEATURES_FOLDER + '/training/ResNet152v2/OneDense512_Dropout_fine_tuning.npy')

In [ ]:
feature_dimension = training_features.shape[1]
feature_dimension

512

In [ ]:
validation_features = np.load(FEATURES_FOLDER + '/validation/ResNet152v2/OneDense512_Dropout_fine_tuning.npy')
test_features = np.load(FEATURES_FOLDER + '/test/ResNet152v2/OneDense512_Dropout_fine_tuning.npy')

In [ ]:
distractor_features = np.load(FEATURES_FOLDER + '/distractor/ResNet152v2/OneDense512_Dropout_fine_tuning.npy')

For feature extraction:

In [5]:
training_features = np.load(FEATURES_FOLDER + '/training/ResNet152v2/OneDense512_Dropout_feature_extraction.npy')
validation_features = np.load(FEATURES_FOLDER + '/validation/ResNet152v2/OneDense512_Dropout_feature_extraction.npy')
test_features = np.load(FEATURES_FOLDER + '/test/ResNet152v2/OneDense512_Dropout_feature_extraction.npy')
distractor_features = np.load(FEATURES_FOLDER + '/distractor/ResNet152v2/OneDense512_Dropout_feature_extraction.npy')

# AutoEncoder Class

In [6]:
class AutoEncoder:
  def __init__(self, train_features, val_features, latent_dim, pace):
    self.train_features = train_features
    self.val_features = val_features
    self.encoder_final_layer = 'Encoder' + str(latent_dim)

    dim = feature_dimension = train_features.shape[1]
    input = keras.Input(shape=(feature_dimension,))
    # Enconder layers
    dim -= pace
    name = 'Encoder' + str(dim)
    x = keras.layers.Dense(dim, activation='relu', name=name)(input)
    while dim != latent_dim:
      dim -= pace
      name = 'Encoder' + str(dim)
      x = keras.layers.Dense(dim, activation='relu', name=name)(x)
    
    # Decoder Layers
    while dim != feature_dimension - pace:
      dim += pace
      name = 'Decoder' + str(dim)
      x = keras.layers.Dense(dim, activation='relu', name=name)(x)
    outputs = keras.layers.Dense(feature_dimension)(x)
    self.model = keras.Model(inputs=input, outputs=outputs)

  def summary(self):
    self.model.summary()

  def compile(self, loss='mean_squared_error', optimizer='sgd'):
    self.model.compile(loss=loss,
                       optimizer=optimizer,
                       metrics=[tf.keras.metrics.CosineSimilarity(axis=1)])
  
  def fit(self, callbacks, epochs=50, patience=5, batch_size=BATCH_SIZE):
    self.history = self.model.fit(self.train_features,
                                  self.train_features,
                                  validation_data=(self.val_features, self.val_features),
                                  callbacks=callbacks,
                                  shuffle=True,
                                  batch_size=batch_size,
                                  epochs=epochs)

  def evaluate(self, test_features):
    test_loss, test_acc = self.model.evaluate((test_features, test_features))
    print(f"Test accuracy: {test_acc:.3f}")

  def predict_features_encoder(self, features, path_name):
    encoder = keras.Model(self.model.input, outputs=self.model.get_layer(self.encoder_final_layer).output)
    features = encoder.predict(features)
    print(features.shape)
    print(features)
    np.save(FEATURES_FOLDER + path_name, features)

  def plot_model(self, model_name):
    keras.utils.plot_model(self.model, model_name, show_shapes=True)

  def plot_loss(self):
      loss = self.history.history['loss']
      val_loss = self.history.history['val_loss']
      plt.plot(range(1, len(loss) + 1), loss, 'r', label='Training Loss')
      plt.plot(range(1, len(loss) + 1), val_loss, 'g', label='Validation Loss')
      plt.title('Training and Validation Loss')
      plt.xlabel('Epochs')
      plt.ylabel('Loss')
      plt.legend()
      plt.plot()

# ResNet 152 512 to 256 features FineTuning

In [ ]:
autoencoder = AutoEncoder(training_features, validation_features, 256, 64)
autoencoder.summary()

Model: "model_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_5 (InputLayer)        [(None, 512)]             0         
                                                                 
 Encoder448 (Dense)          (None, 448)               229824    
                                                                 
 Encoder384 (Dense)          (None, 384)               172416    
                                                                 
 Encoder320 (Dense)          (None, 320)               123200    
                                                                 
 Encoder256 (Dense)          (None, 256)               82176     
                                                                 
 Decoder320 (Dense)          (None, 320)               82240     
                                                                 
 Decoder384 (Dense)          (None, 384)               1232

In [ ]:
autoencoder.plot_model('autoencoder.png')

In [ ]:
autoencoder.compile()
autoencoder.fit(callbacks_list, epochs=800)

Epoch 1/800
185/185 [==============================] - 2s 8ms/step - loss: 0.4580 - cosine_similarity: 0.8981 - val_loss: 0.4299 - val_cosine_similarity: 0.9035
Epoch 2/800
185/185 [==============================] - 1s 7ms/step - loss: 0.4566 - cosine_similarity: 0.8984 - val_loss: 0.4323 - val_cosine_similarity: 0.9029
Epoch 3/800
185/185 [==============================] - 1s 7ms/step - loss: 0.4565 - cosine_similarity: 0.8985 - val_loss: 0.4301 - val_cosine_similarity: 0.9034
Epoch 4/800
185/185 [==============================] - 1s 7ms/step - loss: 0.4561 - cosine_similarity: 0.8986 - val_loss: 0.4279 - val_cosine_similarity: 0.9040
Epoch 5/800
185/185 [==============================] - 1s 7ms/step - loss: 0.4552 - cosine_similarity: 0.8988 - val_loss: 0.4331 - val_cosine_similarity: 0.9028
Epoch 6/800
185/185 [==============================] - 1s 7ms/step - loss: 0.4551 - cosine_similarity: 0.8988 - val_loss: 0.4291 - val_cosine_similarity: 0.9037
Epoch 7/800
185/185 [=============

In [ ]:
autoencoder.predict_features_encoder(training_features, '/training/AutoEncoder/512to256withPace64.npy')

In [ ]:
autoencoder.predict_features_encoder(test_features, '/test/AutoEncoder/512to256withPace64.npy')

In [ ]:
encoder = keras.Model(autoencoder.model.input, outputs=autoencoder.model.get_layer(autoencoder.encoder_final_layer).output)
features = encoder.predict(training_features)
print(features.shape)
print(features)

(47332, 256)
[[ 9.195868   0.         0.        ...  4.9020514  7.052516   7.9435887]
 [ 5.6641865  0.         0.        ...  4.8134475  8.054794  11.099172 ]
 [ 3.6737359  0.         0.        ...  4.0150657  5.6983767  6.2199116]
 ...
 [ 5.750561   0.         0.        ...  5.7983074  8.606042   4.752095 ]
 [ 5.8153877  0.         0.        ...  6.437734   9.22006    5.7890897]
 [ 6.7102695  0.         0.        ...  5.274567   9.734262   3.4280245]]


In [ ]:
np.save(FEATURES_FOLDER + '/training/AutoEncoder/512to256withPace64.npy', features)

In [ ]:
features = encoder.predict(test_features)
print(features.shape)
print(features)
np.save(FEATURES_FOLDER + '/test/AutoEncoder/512to256withPace64.npy', features)

(1625, 256)
[[ 4.312674   0.         0.        ...  6.7134914  4.238811   7.0304623]
 [ 3.7251554  0.         0.        ...  4.957602   2.7158318  7.8004165]
 [ 7.5487733  0.         0.        ...  9.545107   9.403517  11.842505 ]
 ...
 [ 6.1560497  0.         0.        ...  4.4695544  9.040167   4.426116 ]
 [ 1.931897   0.         0.        ...  6.401012   9.4310665  3.4554417]
 [ 7.1203866  0.         0.        ...  3.9933767 10.865172   6.2109866]]


In [ ]:
features = encoder.predict(distractor_features)
print(features.shape)
print(features)
np.save(FEATURES_FOLDER + '/distractor/AutoEncoder/512to256withPace64.npy', features)

(25000, 256)
[[ 2.4274487   0.          0.         ...  2.806142    2.5323086
   6.6855717 ]
 [ 5.010743    0.          0.         ...  4.1745405   3.1924844
   4.3320932 ]
 [ 0.03499752  0.          0.         ...  6.329656    5.6457787
   2.8367631 ]
 ...
 [ 3.4716578   0.          0.         ...  5.598138    4.1397676
   5.058101  ]
 [ 6.2002625   0.          0.         ...  2.197337    4.8735094
   4.8600874 ]
 [ 4.163387    0.          0.         ...  0.         12.308101
   7.404747  ]]


In [ ]:
! cp autoencoder.keras /content/drive/MyDrive/CV_Birds/models/AutoEncoder/

# ResNet152v2 512 to 128 Finetuning

In [ ]:
autoencoder = AutoEncoder(training_features, validation_features, 128, 64)
autoencoder.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 512)]             0         
                                                                 
 Encoder448 (Dense)          (None, 448)               229824    
                                                                 
 Encoder384 (Dense)          (None, 384)               172416    
                                                                 
 Encoder320 (Dense)          (None, 320)               123200    
                                                                 
 Encoder256 (Dense)          (None, 256)               82176     
                                                                 
 Encoder192 (Dense)          (None, 192)               49344     
                                                                 
 Encoder128 (Dense)          (None, 128)               24704 

In [ ]:
autoencoder.plot_model('autoencoder.png')

In [ ]:
autoencoder.compile()
autoencoder.fit(callbacks_list, epochs=800)

Epoch 1/800
185/185 [==============================] - 3s 10ms/step - loss: 0.5476 - cosine_similarity: 0.8767 - val_loss: 0.5153 - val_cosine_similarity: 0.8830
Epoch 2/800
185/185 [==============================] - 2s 9ms/step - loss: 0.5472 - cosine_similarity: 0.8768 - val_loss: 0.5133 - val_cosine_similarity: 0.8835
Epoch 3/800
185/185 [==============================] - 2s 9ms/step - loss: 0.5470 - cosine_similarity: 0.8769 - val_loss: 0.5193 - val_cosine_similarity: 0.8821
Epoch 4/800
185/185 [==============================] - 2s 9ms/step - loss: 0.5466 - cosine_similarity: 0.8769 - val_loss: 0.5167 - val_cosine_similarity: 0.8827
Epoch 5/800
185/185 [==============================] - 2s 9ms/step - loss: 0.5458 - cosine_similarity: 0.8772 - val_loss: 0.5213 - val_cosine_similarity: 0.8816
Epoch 6/800
185/185 [==============================] - 2s 9ms/step - loss: 0.5458 - cosine_similarity: 0.8771 - val_loss: 0.5130 - val_cosine_similarity: 0.8835
Epoch 7/800
185/185 [============

In [ ]:
autoencoder.predict_features_encoder(training_features, '/training/AutoEncoder/512to128withPace64.npy')

(47332, 128)
[[ 9.49873    0.         3.6662238 ...  3.4222531  9.934501   5.38992  ]
 [ 8.82283    0.         7.693001  ...  1.2568046  7.6079645  3.764613 ]
 [ 8.100039   0.         3.3028147 ...  2.8523085  8.390544   5.5698853]
 ...
 [ 2.8285484  0.         9.807598  ...  8.710649   8.182634   6.107458 ]
 [ 5.7676873  0.         9.587541  ... 11.275144   8.428513   8.149769 ]
 [ 2.8090334  0.         7.974751  ...  8.699295   5.222485   9.901351 ]]


In [ ]:
autoencoder.predict_features_encoder(test_features, '/test/AutoEncoder/512to128withPace64.npy')

(1625, 128)
[[ 6.179785   0.         5.780048  ...  2.2905853  9.690669   6.5708756]
 [ 5.78261    0.         6.6455693 ...  2.225507   9.475985   5.7978067]
 [18.115143   0.         7.341868  ...  3.4316201 16.21714   11.6665745]
 ...
 [ 3.4109862  0.         8.680737  ...  7.7216544  7.7940617  6.8752756]
 [ 7.2186117  0.         3.9433327 ...  9.629163   7.211761  11.431813 ]
 [ 4.1829176  0.         6.233928  ... 10.220056   9.355988  10.461566 ]]


In [ ]:
autoencoder.predict_features_encoder(distractor_features, '/distractor/AutoEncoder/512to128withPace64.npy')

(25000, 128)
[[ 5.8749156  0.         6.843479  ...  3.342217   7.203499   4.9853525]
 [ 4.136994   0.         5.7078066 ...  3.9104564  6.2266994  4.3197236]
 [ 8.090802   0.        10.103582  ...  4.478878  10.942012   3.4928403]
 ...
 [ 4.252955   0.         5.00579   ...  2.8811474  6.818927   2.8188703]
 [ 5.1878843  0.         8.587541  ...  6.5327635 10.593627   0.5781613]
 [ 9.551695   0.        30.397394  ...  9.791951  14.976327   3.795782 ]]


In [ ]:
! cp autoencoder128.keras /content/drive/MyDrive/CV_Birds/models/AutoEncoder/

# ResNet152v2 512 to 256 Feature Extraction

In [7]:
autoencoder = AutoEncoder(training_features, validation_features, 256, 64)
autoencoder.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 512)]             0         
                                                                 
 Encoder448 (Dense)          (None, 448)               229824    
                                                                 
 Encoder384 (Dense)          (None, 384)               172416    
                                                                 
 Encoder320 (Dense)          (None, 320)               123200    
                                                                 
 Encoder256 (Dense)          (None, 256)               82176     
                                                                 
 Decoder320 (Dense)          (None, 320)               82240     
                                                                 
 Decoder384 (Dense)          (None, 384)               123264

In [ ]:
autoencoder.plot_model('autoencoder.png')

In [9]:
autoencoder.compile()
autoencoder.fit(callbacks_list, epochs=800)

Epoch 1/800
185/185 [==============================] - 2s 8ms/step - loss: 0.4754 - cosine_similarity: 0.8978 - val_loss: 0.4617 - val_cosine_similarity: 0.9031
Epoch 2/800
185/185 [==============================] - 1s 7ms/step - loss: 0.4747 - cosine_similarity: 0.8979 - val_loss: 0.4579 - val_cosine_similarity: 0.9039
Epoch 3/800
185/185 [==============================] - 2s 9ms/step - loss: 0.4741 - cosine_similarity: 0.8981 - val_loss: 0.4513 - val_cosine_similarity: 0.9054
Epoch 4/800
185/185 [==============================] - 1s 8ms/step - loss: 0.4739 - cosine_similarity: 0.8981 - val_loss: 0.4555 - val_cosine_similarity: 0.9044
Epoch 5/800
185/185 [==============================] - 1s 7ms/step - loss: 0.4740 - cosine_similarity: 0.8981 - val_loss: 0.4592 - val_cosine_similarity: 0.9037
Epoch 6/800
185/185 [==============================] - 1s 8ms/step - loss: 0.4721 - cosine_similarity: 0.8985 - val_loss: 0.4513 - val_cosine_similarity: 0.9054
Epoch 7/800
185/185 [=============

In [10]:
autoencoder.predict_features_encoder(training_features, '/training/AutoEncoder/512to256withPace64_feature_extraction.npy')

(47332, 256)
[[ 0.         7.85741    4.8072505 ...  0.        11.526674  10.304251 ]
 [ 0.         5.131463   7.307387  ...  0.        10.300343  13.863658 ]
 [ 0.         4.4632373  4.5082455 ...  0.         8.203612   6.196382 ]
 ...
 [ 0.        10.703678   1.8268989 ...  0.         3.1073825  5.9998307]
 [ 0.         9.101698   5.8591743 ...  0.         8.225212   7.0080075]
 [ 0.         8.41263    4.068163  ...  0.         4.482814   7.8149524]]


In [11]:
autoencoder.predict_features_encoder(test_features, '/test/AutoEncoder/512to256withPace64_feature_extraction.npy')

(1625, 256)
[[ 0.         4.8632526  7.103323  ...  0.         6.8853297  6.4178576]
 [ 0.         6.0288143  4.880917  ...  0.         7.0359607  5.3576922]
 [ 0.         6.29494    7.314261  ...  0.        13.789407  11.367596 ]
 ...
 [ 0.        10.634909   2.7248502 ...  0.         3.6018918  7.557374 ]
 [ 0.         5.810537   3.629338  ...  0.         2.3764591  3.8160381]
 [ 0.         6.1706696  3.2348762 ...  0.         4.1383724  4.807791 ]]


In [12]:
autoencoder.predict_features_encoder(distractor_features, '/distractor/AutoEncoder/512to256withPace64_feature_extraction.npy')

(25000, 256)
[[ 0.          7.325205    4.155388   ...  0.          9.071452
   8.184651  ]
 [ 0.          7.768376    4.2125835  ...  0.          7.9366603
  10.824646  ]
 [ 0.02880119  7.7105      6.4099507  ...  0.          8.799576
   5.110255  ]
 ...
 [ 0.          8.310111    3.156978   ...  0.          8.130999
   6.8294177 ]
 [ 0.          4.820765    4.145028   ...  0.          7.820896
   3.5790474 ]
 [ 0.         11.396017    7.3049135  ...  0.         13.513865
  12.441708  ]]


In [13]:
! cp autoencoder256_feature_extraction.keras /content/drive/MyDrive/CV_Birds/models/AutoEncoder/

# ResNet152v2 512 to 128 Feature Extraction

In [14]:
autoencoder = AutoEncoder(training_features, validation_features, 128, 64)
autoencoder.summary()

Model: "model_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 512)]             0         
                                                                 
 Encoder448 (Dense)          (None, 448)               229824    
                                                                 
 Encoder384 (Dense)          (None, 384)               172416    
                                                                 
 Encoder320 (Dense)          (None, 320)               123200    
                                                                 
 Encoder256 (Dense)          (None, 256)               82176     
                                                                 
 Encoder192 (Dense)          (None, 192)               49344     
                                                                 
 Encoder128 (Dense)          (None, 128)               2470

In [ ]:
autoencoder.plot_model('autoencoder.png')

In [18]:
autoencoder.compile()
autoencoder.fit(callbacks_list, epochs=800)

Epoch 1/800
185/185 [==============================] - 3s 10ms/step - loss: 0.6413 - cosine_similarity: 0.8594 - val_loss: 0.6210 - val_cosine_similarity: 0.8674
Epoch 2/800
185/185 [==============================] - 2s 9ms/step - loss: 0.6407 - cosine_similarity: 0.8596 - val_loss: 0.6122 - val_cosine_similarity: 0.8693
Epoch 3/800
185/185 [==============================] - 2s 9ms/step - loss: 0.6401 - cosine_similarity: 0.8597 - val_loss: 0.6168 - val_cosine_similarity: 0.8683
Epoch 4/800
185/185 [==============================] - 2s 9ms/step - loss: 0.6383 - cosine_similarity: 0.8601 - val_loss: 0.6176 - val_cosine_similarity: 0.8681
Epoch 5/800
185/185 [==============================] - 2s 8ms/step - loss: 0.6378 - cosine_similarity: 0.8602 - val_loss: 0.6149 - val_cosine_similarity: 0.8687
Epoch 6/800
185/185 [==============================] - 2s 8ms/step - loss: 0.6364 - cosine_similarity: 0.8605 - val_loss: 0.6127 - val_cosine_similarity: 0.8692
Epoch 7/800
185/185 [============

In [19]:
autoencoder.predict_features_encoder(training_features, '/training/AutoEncoder/512to128withPace64_feature_extraction.npy')

(47332, 128)
[[ 0.         0.         0.        ...  8.075813  16.673262  13.597744 ]
 [ 0.         0.         0.        ... 16.640043  15.179927  12.970896 ]
 [ 0.         0.         0.        ...  9.167509   9.841227  10.882952 ]
 ...
 [ 0.         0.         0.        ...  8.19948    4.7598386 14.980965 ]
 [ 0.         0.         0.        ...  7.0964785  7.913847  11.806072 ]
 [ 0.         0.         0.        ...  7.9192142  5.543435  13.611256 ]]


In [20]:
autoencoder.predict_features_encoder(test_features, '/test/AutoEncoder/512to128withPace64_feature_extraction.npy')

(1625, 128)
[[ 0.         0.         0.        ... 12.095595  10.663634  12.708031 ]
 [ 0.         0.         0.        ... 10.724072  13.293723  10.43426  ]
 [ 0.         0.         0.        ... 13.318824  15.602181  14.75976  ]
 ...
 [ 0.         0.         0.        ...  6.22002    7.3831186 13.982892 ]
 [ 0.         0.         0.        ...  5.13063    4.323975  13.33562  ]
 [ 0.         0.         0.        ...  5.5317206  5.7060337 15.30135  ]]


In [21]:
autoencoder.predict_features_encoder(distractor_features, '/distractor/AutoEncoder/512to128withPace64_feature_extraction.npy')

(25000, 128)
[[ 0.         0.         0.        ...  9.859884  11.114941   8.086099 ]
 [ 0.         0.         0.        ...  5.585647  14.122535   9.786284 ]
 [ 0.         0.         0.        ...  5.7548776  5.637664   6.853354 ]
 ...
 [ 0.         0.         0.        ...  6.9875574  7.9005203  4.8430367]
 [ 0.         0.         0.        ...  9.459765  10.160999   3.7016633]
 [ 0.         0.         0.        ... 20.241228  16.594048   4.9529614]]


In [23]:
keras.models.save_model(autoencoder.model, '/content/drive/MyDrive/CV_Birds/models/AutoEncoder/autoencoder128_feature_extraction.keras')

In [ ]:
! cp autoencoder128_feature_extraction.keras /content/drive/MyDrive/CV_Birds/models/AutoEncoder/